In [194]:
from PIL import Image
import sys, os
import numpy as np
import imageio
from scipy import ndimage
import json
from enum import Enum
from typing import Dict

In [215]:
class Attribute(Enum):
    body = 1
    bottomwear = 2
    eyes = 3
    hair = 4
    shoes = 5
    topwear = 6
    action = 7

attributes = {
    Attribute.body: {
        "0": "man with skin of color reddish orange",
        "1": "man with skin of color yellowish orange",
        "2": "man with skin of color tealish blue",
        "3": "pale man with white skin",
        "4": "man with skin of color dark orange",
        "5": "man with brown skin ",
        "6": "orc with green skin" 
    },
    Attribute.bottomwear: {
        "0": "white shorts",
        "1": "leather pants",
        "2": "red shorts",
        "3": "white pants",
        "4": "light green shorts",
        "5": "dark green pants",
        "6": "leather sash"
    },
    Attribute.eyes: {
        "0": "blue eyes",
        "1": "brown eyes",
        "2": "red eyes",
        "3": "green eyes",
        "4": "yellow eyes"
    },
    Attribute.hair : {
        "0": "green short hair",
        "1": "man bun purple hair",
        "2": "yellow short hair",
        "3": "gray short hair",
        "4": "pinkis red short hair",
        "5": "pinkish purple curly hair",
        "6": "gray halfshaven hair",
        "7": "red short hair",
        "8": "pink short hair",
        "9": "orange curly hair"
    },
    Attribute.shoes: {
        "0": "brown shoes",
        "1": "yellow shoes",
        "2": "white shoes"
    },
    Attribute.topwear: {
        "0": "red shirt", 
        "1": "blue shirt", 
        "2": "white shirt",
        "3": "gray armor",
        "4": "leather armor",
        "5": "white formal shirt with tie",
        "6": "gray chainmail"
    }
}

actions = {
    'casting a spell': {
        'back view': list(range(0, 7)),
        'left side view': list(range(13, 20)),
        'front view': list(range(26, 33)),
        'right-sided view': list(range(39, 46))
    },
    'thrusting': {
        'back view': list(range(52, 60)),
        'left': list(range(65, 73)),
        'front view': list(range(78, 86)),
        'right-sided view': list(range(91, 99))
    },
    'walking': {
        'back view': list(range(104, 113)),
        'left side view': list(range(117, 126)),
        'front view': list(range(130, 139)),
        'right-sided view': list(range(143, 152))
    },
    'slashing': {
        'back view': list(range(156, 162)),
        'left side view': list(range(169, 175)),
        'front view': list(range(182, 188)),
        'right-sided view': list(range(195, 201))
    },
    'shooting': {
        'back view': list(range(208, 221)),
        'left side view': list(range(221, 234)),
        'front view': list(range(234, 247)),
        'right-sided view': list(range(247, 260))
    },
    'falling': {'front view': list(range(260, 266))}
}

# create dictionary of all attribute variants; structure: atrr_img = {'body': {'0': Img, ...}, ...}
attr_img:Dict[Attribute, Dict[str, Image.Image]] = {} 
for attr_name, attr_values in attributes.items():
    attr_img[attr_name] = {}
    for name in attr_values:
        img_path = os.path.join(attr_name.name, f"{name}.png")
        attr_img[attr_name][name] = Image.open(img_path)

In [216]:
def get_random_char() -> Dict[Attribute, Dict[str, object]]:
    character = {}
    for k, v in attributes.items():
        rnd_key = np.random.choice(list(v.keys()))
        character[k] = {rnd_key: v[rnd_key]}
    
    rnd_action_key = np.random.choice(list(actions.keys()))
    rnd_action_val = actions[rnd_action_key]
    
    rnd_side_key = np.random.choice(list(rnd_action_val.keys()))
    rnd_side_val = rnd_action_val[rnd_side_key]
    
    character[Attribute.action] = {rnd_action_key: {rnd_side_key: rnd_side_val}}
    return character

random_characters = [get_random_char() for i in range(1)]
random_characters            

[{<Attribute.body: 1>: {'6': 'orc with green skin'},
  <Attribute.bottomwear: 2>: {'3': 'white pants'},
  <Attribute.eyes: 3>: {'4': 'yellow eyes'},
  <Attribute.hair: 4>: {'4': 'pinkis red short hair'},
  <Attribute.shoes: 5>: {'1': 'yellow shoes'},
  <Attribute.topwear: 6>: {'6': 'gray chainmail'},
  <Attribute.action: 7>: {'slashing': {'right-sided view': [195,
     196,
     197,
     198,
     199,
     200]}}}]

In [222]:
def firstKey(dict)-> str:
    return next(iter(dict.items()))[0]

for character in random_characters:
    # ugly af
    body_img = attr_img[Attribute.body][firstKey(character[Attribute.body])]
    bottomwear_img = attr_img[Attribute.bottomwear][firstKey(character[Attribute.bottomwear])] 
    eyes_img = attr_img[Attribute.eyes][firstKey(character[Attribute.eyes])] 
    hair_img = attr_img[Attribute.hair][firstKey(character[Attribute.hair])] 
    shoes_img = attr_img[Attribute.shoes][firstKey(character[Attribute.shoes])]
    topwear_img = attr_img[Attribute.topwear][firstKey(character[Attribute.topwear])] 
    imgs = [body_img, bottomwear_img, eyes_img, hair_img, shoes_img, topwear_img]
    action = character[Attribute.action]
    action_desc, frames_dict = next(iter(action.items())) 
    side_desc, frames = next(iter(frames_dict.items())) 
    
    frame_size = 64
    max_frames_per_row = 13
    frame_row = int(frames[0] / max_frames_per_row)
    for i, frame in enumerate(frames):
        box = (i * frame_size, frame_row * frame_size, (i + 1) * frame_size, (frame_row + 1) * frame_size)
        body_img.crop()
    
    print(f"{action_desc=}, {side_desc=}, {frames=}, {frame_row=}")
    # for char_attr, dict in character.items():
    #     name, desc = next(iter(dict.items()))
    #     print(char_attr, name, desc)
    #     if char_attr == "action":
    #         desc, frame_indexes = next(iter(desc.items()))
    #         print(f"action, {name} {desc}, {frame_indexes}")
    #     else:

action_desc='slashing', side_desc='right-sided view', frames=[195, 196, 197, 198, 199, 200], frame_row=15


In [ ]:

img_list = []
frames_dir = "frames"
data_dir = "data"
images_dir = "images"
metadata_path = os.path.join(data_dir, "metadata.jsonl");
data_images_dir = os.path.join(data_dir, "images")
states = [("slash", "slashing", 8), ("spellcard", "casting a spell", 8), ("walk", "walking", 8)]
sides = [("front", "front view"), ("left", "left-sided view"), ("right", "right sided view")]
character = ["green", "violet", "orange", "gray", "red", "pink"]
size = 128
jsoncontent = []

if not os.path.exists(data_images_dir):
    os.makedirs(data_images_dir)

for n, char_attribute in enumerate(character):
    for (state, state_desc, n_frames) in states: 
        for (side, side_desc) in sides:
            sequence = Image.new("RGBA", (size*8, size*8), 'black')
            for i in range(n_frames):
                img_file = f"{side}_{n:04d}_{i}.png"
                frame_path = os.path.join(frames_dir, state, img_file);
                frame = Image.open(frame_path);
                frame = frame.resize((128,128))
                sequence.paste(frame, (i * size, 0))
            
            seq_name =  f"{n:04d}_{state}_{side}.png"
            seq_path = os.path.join(data_images_dir, seq_name);
            jsoncontent.append((f"{images_dir}/{seq_name}", f"sprite animation showing a {char_attribute} haired man {state_desc}, composed of {n_frames} frames, {side_desc}"))
            sequence.save(seq_path)

metadata = [{"file_name": item[0], "text": item[1]} for item in jsoncontent]

with open(metadata_path, "w") as jsonfile:
    for item in metadata:
        json.dump(item, jsonfile)
        jsonfile.write('\n')

            